## CFM 301 DA4
### Q6
#### Jeongseop Yi (Patrick), j22yi

In [17]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels.api as sm

In [18]:
q6_df = pd.read_sas('q6_data.sas7bdat')
q6_df['date'] = pd.to_datetime(q6_df['DATE'])
q6_df['year'] = q6_df['date'].dt.year
q6_df['month'] = q6_df['date'].dt.month
q6_df['yyyymm'] = q6_df['year'].astype(str) + q6_df['month'].astype(str).str.zfill(2)

q6_df.columns = list(map(str.lower, q6_df.columns))
q6_df = q6_df.drop_duplicates(subset=['permno', 'yyyymm'])
q6_df['prc'] = q6_df['prc'].abs()
q6_df['mktcap'] = q6_df['prc'] * q6_df['shrout']
q6_df['lnSize'] = np.log(q6_df['mktcap'])
q6_df['prc_prev'] = q6_df.groupby(['permno', 'shrcd', 'exchcd'])['prc'].shift(1)
q6_df['mktcap_prev'] = q6_df.groupby(['permno', 'shrcd', 'exchcd'])['mktcap'].shift(1)
q6_df['ret_t1'] = q6_df.groupby(['permno', 'shrcd', 'exchcd'])['ret'].shift(-1)

q6_df = q6_df[(q6_df['mktcap_prev'] >= 100 * 1000) & (q6_df['prc_prev'] >= 5)]

In [19]:
# winsorize log market cap by 3 standard deviations
q6_df['lnSize_w'] = q6_df.groupby('yyyymm')['lnSize'].transform(lambda x: x.clip(x.mean() - 3 * x.std(), x.mean() + 3 * x.std()))

In [20]:
def get_monthly_returns_quintile(df, col):
    df[col + '_q'] = df.groupby('yyyymm')[col].transform(lambda x: pd.qcut(x, 5, labels=False))
    df[col + '_q_ret'] = df.groupby(['yyyymm', col + '_q'])['ret_t1'].transform('mean')
    # create data frame with monthly returns for each quintile
    df_q_ret = pd.DataFrame(columns=['1', '2', '3', '4', '5'])
    for i in range(1, 6):
        df_q_ret[str(i)] = df[df[col + '_q'] == i - 1].groupby('yyyymm')[col + '_q_ret'].mean()
    df_q_ret = df_q_ret.reset_index()
    return df, df_q_ret

In [21]:
factors = ['lnSize_w']
factors_quinlst = []
q6_df.set_index('yyyymm', inplace=True)
for factor in factors:
    q6_df, factor_ret = get_monthly_returns_quintile(q6_df, factor)
    factors_quinlst.append(factor_ret)
q6_df.reset_index(inplace=True)

In [22]:
for i in range(len(factors)):
    factors_quinlst[i]['port'] = factors_quinlst[i]['1'] - factors_quinlst[i]['5']

In [23]:
factors_port = pd.DataFrame(columns=['yyyymm'].append(factors))
factors_port['yyyymm'] = factors_quinlst[0]['yyyymm']
for i in range(len(factors)):
    factors_port[factors[i]] = factors_quinlst[i]['port']

In [24]:
ff4_df = pd.read_sas('ff4.sas7bdat')
ff4_df.columns = map(str.lower, ff4_df.columns)
ff4_df['date'] = pd.to_datetime(ff4_df['dateff'])
ff4_df['year'] = ff4_df['date'].dt.year
ff4_df['month'] = ff4_df['date'].dt.month
ff4_df['yyyymm'] = ff4_df['year'].astype(str) + ff4_df['month'].astype(str).str.zfill(2)
ff4_df = ff4_df[(ff4_df['yyyymm'] >= '200001') & (ff4_df['yyyymm'] <= '202111')]

factors_port = pd.merge(factors_port, ff4_df, on='yyyymm', how='left')
factors_port = factors_port[(factors_port['yyyymm'] >= '200001') & (factors_port['yyyymm'] <= '202111')]

In [25]:
summary_df = pd.DataFrame(columns=['index'].append(factors))
summary_df['index'] = ['overall return', 'excess return',
                       'CAPM alpha', 'CAPM t-stat', 'FF4 alpha', 'FF4 t-stat', 'Sharpe']
summary_df.set_index('index', inplace=True)
for factor in factors:
    mkt = sm.add_constant(factors_port['mktrf'])
    model = sm.OLS(factors_port[factor] - factors_port['rf'], mkt).fit()
    model_alpha = model.params[0]
    model_tstat = model.tvalues[0]

    ff4 = sm.add_constant(factors_port[['mktrf', 'smb', 'hml', 'umd']])
    model_ff4 = sm.OLS(factors_port[factor] - factors_port['rf'], ff4).fit()
    model_ff4_alpha = model_ff4.params[0]
    model_ff4_tstat = model_ff4.tvalues[0]

    summary_df[factor] = [factors_port[factor].mean(),
                          (factors_port[factor] - factors_port['rf'] -
                           factors_port['mktrf']).mean(),
                          model_alpha,
                          model_tstat.round(2),
                          model_ff4_alpha,
                          model_ff4_tstat.round(2),
                          (factors_port[factor] - factors_port['rf']).mean() / (factors_port[factor] - factors_port['rf']).std() * np.sqrt(12)]

display(summary_df.round(4).T)


index     overall return  excess return  CAPM alpha  CAPM t-stat  FF4 alpha  \
lnSize_w          0.0011        -0.0061     -0.0008         -0.4    -0.0008   

index     FF4 t-stat  Sharpe  
lnSize_w        -0.4 -0.0111

In [27]:
q6_df_summary = q6_df.describe(percentiles=[0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99])
display(q6_df_summary)
with pd.ExcelWriter("../DA4_data.xlsx", mode='a', engine="openpyxl", if_sheet_exists='replace') as writer:
    q6_df_summary.to_excel(writer, sheet_name="q6_summary", index=False)


permno          shrcd         exchcd            prc  \
count  849654.000000  849654.000000  849654.000000  845505.000000   
mean    62733.916680      11.090891       2.065194      89.906455   
std     29830.219563       0.289092       0.997873    3399.348677   
min     10002.000000      10.000000       1.000000       0.180000   
1%      10395.000000      11.000000       1.000000       4.850000   
5%      12236.000000      11.000000       1.000000       6.625000   
25%     29111.000000      11.000000       1.000000      13.500000   
50%     78451.000000      11.000000       3.000000      23.950001   
75%     86814.000000      11.000000       3.000000      41.650002   
95%     91579.000000      12.000000       3.000000      97.430000   
99%     93071.000000      12.000000       3.000000     237.169198   
max     93436.000000      12.000000       3.000000  450662.000000   

                 ret        shrout           year          month  \
count  845464.000000  8.490020e+05  849654.000000  849654.000000   
mean        0.008755  1.483237e+05    2009.882593       6.523187   
std         0.145811  4.789766e+05       6.774683       3.436209   
min        -0.981295  1.910000e+02    1999.000000       1.000000   
1%         -0.371613  4.674000e+03    1999.000000       1.000000   
5%         -0.201249  8.721000e+03    2000.000000       1.000000   
25%        -0.056589  2.150100e+04    2004.000000       4.000000   
50%         0.006098  4.273950e+04    2010.000000       7.000000   
75%         0.069079  1.041010e+05    2016.000000      10.000000   
95%         0.218865  5.341290e+05    2021.000000      12.000000   
99%         0.429226  1.830396e+06    2021.000000      12.000000   
max        19.883589  2.288030e+07    2021.000000      12.000000   

             mktcap         lnSize       prc_prev   mktcap_prev  \
count  8.455050e+05  845505.000000  849654.000000  8.496540e+05   
mean   6.720132e+06      14.037542      89.010519  6.651067e+06   
std    3.080728e+07       1.623793    3356.346567  3.025043e+07   
min    2.314750e+03       7.747057       5.000000  1.000000e+05   
1%     1.019451e+05      11.532190       5.350000  1.072772e+05   
5%     1.369081e+05      11.827065       6.810000  1.387079e+05   
25%    3.510197e+05      12.768598      13.560000  3.507730e+05   
50%    9.900971e+05      13.805558      23.910000  9.825722e+05   
75%    3.421791e+06      15.045675      41.470001  3.388449e+06   
95%    2.635323e+07      17.087101      96.599998  2.606623e+07   
99%    1.075952e+08      18.493887     233.223498  1.066271e+08   
max    2.902368e+09      21.788793  436000.000000  2.711977e+09   

              ret_t1       lnSize_w     lnSize_w_q  lnSize_w_q_ret  
count  836857.000000  845505.000000  845505.000000   841501.000000  
mean        0.008781      14.034749       1.999849        0.008781  
std         0.143923       1.615299       1.414455        0.058146  
min        -0.981295       9.202905       0.000000       -0.253231  
1%         -0.375000      11.532190       0.000000       -0.146372  
5%         -0.201669      11.827065       0.000000       -0.091087  
25%        -0.056376      12.768598       1.000000       -0.024002  
50%         0.006307      13.805558       2.000000        0.012477  
75%         0.069343      15.045675       3.000000        0.044001  
95%         0.218910      17.087101       4.000000        0.093909  
99%         0.429561      18.350303       4.000000        0.160152  
max        16.250530      19.850943       4.000000        0.202709

Exception ignored in: <function ZipFile.__del__ at 0x7f4de76cc280>
Traceback (most recent call last):
  File "/usr/lib/python3.10/zipfile.py", line 1819, in __del__
    self.close()
  File "/usr/lib/python3.10/zipfile.py", line 1836, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file
